<a href="https://colab.research.google.com/github/tcglarry/kg_power_line/blob/master/vsb_power_line_eda_correct_3_axis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# Step 1
# Install a Drive FUSE wrapper.
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse


E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 110842 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.1-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


In [2]:

# Step 2
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
!pip install pyarrow

    100% |████████████████████████████████| 11.6MB 2.8MB/s 


In [0]:
!sudo chmod 444 'drive/My Drive/kaggle/vsb_power_line/data/train.parquet'

In [0]:
!cp 'drive/My Drive/kaggle/vsb_power_line/data/train.parquet' .

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pyarrow.parquet as pq
import matplotlib as mpl
mpl.style.use('default')
from PIL import Image
import seaborn as sns
%matplotlib inline

In [0]:
src= 'drive/My Drive/kaggle/vsb_power_line/data/'
src_train_idx_1 = 'drive/My Drive/kaggle/vsb_power_line/data/train_test_data_2/train/1/'
src_train_idx_0 = 'drive/My Drive/kaggle/vsb_power_line/data/train_test_data_2/train/0/'
src_valid_idx_1 = 'drive/My Drive/kaggle/vsb_power_line/data/train_test_data_2/validation/1/'
src_valid_idx_0 = 'drive/My Drive/kaggle/vsb_power_line/data/train_test_data_2/validation/0/'
src_test = 'drive/My Drive/kaggle/vsb_power_line/data/train_test_data_2/test/'

In [8]:
train_df = pd.read_csv(src+'metadata_train.csv')
print (train_df.shape)
train_df['target'].value_counts()
train_df.head()

(8712, 4)


,signal_id,id_measurement,phase,target
0,0,0,0,0
1,1,0,1,0
2,2,0,2,0
3,3,1,0,1
4,4,1,1,1


In [9]:
def every_3_sum_label(x):
    #print (x)
    length = x.shape[0]
    print ('len', length)
    y = x.copy()
    for i in range(length//3):

        y[i*3:i*3+3] = x[i*3:i*3+3].sum()

    return y
  
train_df['re_target'] = every_3_sum_label(train_df['target'])
train_df.head()

len 8712


,signal_id,id_measurement,phase,target,re_target
0,0,0,0,0,0
1,1,0,1,0,0
2,2,0,2,0,0
3,3,1,0,1,3
4,4,1,1,1,3


In [0]:
def every_3_tuple(x):
  
  y_list = []
  #print (x)
  length = x.shape[0]
  print ('len', length)
  y = x.copy()
  for i in range(length//3):

    #print (x[i*3],x[i*3+1] ,x[i*3+2])




    #y[i*3:i*3+3] = (x[i*3],x[i*3+1] ,x[i*3+2])*3
    
    
    y_list.append((x[i*3],x[i*3+1] ,x[i*3+2]))
    
    y_list.append((x[i*3],x[i*3+1] ,x[i*3+2]))
    
    y_list.append((x[i*3],x[i*3+1] ,x[i*3+2]))
    
  print (len(y_list))


  return y_list
  
train_df['target_tuple'] = every_3_tuple(train_df['target'])
train_df.head()

len 8712
8712


,signal_id,id_measurement,phase,target,re_target,target_tuple
0,0,0,0,0,0,"(0, 0, 0)"
1,1,0,1,0,0,"(0, 0, 0)"
2,2,0,2,0,0,"(0, 0, 0)"
3,3,1,0,1,3,"(1, 1, 1)"
4,4,1,1,1,3,"(1, 1, 1)"


In [0]:

train_df['target_tuple'].value_counts()

(0, 0, 0)    8130
(1, 1, 1)     468
(0, 0, 1)      36
(1, 0, 1)      30
(1, 0, 0)      18
(1, 1, 0)      18
(0, 1, 1)       9
(0, 1, 0)       3
Name: target_tuple, dtype: int64

In [0]:
train_df['categorical'] = train_df['target_tuple']
x_dict ={}
idx = list(np.arange(8))
v = list(np.unique(train_df['categorical']))



for i,j in enumerate(v):
  
  print (j)
  
  print (idx[i])
  x_dict[j] = idx[i]

print (x_dict)


(0, 0, 0)
0
(0, 0, 1)
1
(0, 1, 0)
2
(0, 1, 1)
3
(1, 0, 0)
4
(1, 0, 1)
5
(1, 1, 0)
6
(1, 1, 1)
7
{(0, 0, 0): 0, (0, 0, 1): 1, (0, 1, 0): 2, (0, 1, 1): 3, (1, 0, 0): 4, (1, 0, 1): 5, (1, 1, 0): 6, (1, 1, 1): 7}


In [0]:
train_df['categorical'] = train_df['categorical'].map(x_dict)

In [0]:
train_df

,signal_id,id_measurement,phase,target,re_target,target_tuple,categorical
0,0,0,0,0,0,"(0, 0, 0)",0
1,1,0,1,0,0,"(0, 0, 0)",0
2,2,0,2,0,0,"(0, 0, 0)",0
3,3,1,0,1,3,"(1, 1, 1)",7
4,4,1,1,1,3,"(1, 1, 1)",7
5,5,1,2,1,3,"(1, 1, 1)",7
6,6,2,0,0,0,"(0, 0, 0)",0
7,7,2,1,0,0,"(0, 0, 0)",0
8,8,2,2,0,0,"(0, 0, 0)",0
9,9,3,0,0,0,"(0, 0, 0)",0


In [0]:
train_df.to_csv(src+'new_train.csv', index=False)

In [0]:
train_df['categorical'].value_counts()

0    8130
7     468
1      36
5      30
6      18
4      18
3       9
2       3
Name: categorical, dtype: int64

In [0]:
choose=[i for i in range(8712) if i%3 ==0]


In [0]:
id_label = train_df.loc[choose,['id_measurement','categorical']]
id_label

,id_measurement,categorical
0,0,0
3,1,7
6,2,0
9,3,0
12,4,0
15,5,0
18,6,0
21,7,0
24,8,0
27,9,0


In [0]:
id_label.to_csv(src+'id_label.csv',index=False)

In [0]:
df = pd.read_csv(src+'id_label.csv')
train_list = [0,1,2,3,4,5,6,7]
non_valid_list = [2,3]
def select_dict():
    select={}
    
    for i in train_list: 
        if i not in non_valid_list:
            label = df.loc[df['categorical']==i,'id_measurement'].tolist()
            label_valid =  list(np.random.choice(label, int(len(label)*0.2)))
            
            label_train = [i for i in label if i not in label_valid]
            
            
            
            select[str(i)] = [label_train,label_valid]
            
        else:
            label = df.loc[df['categorical']==i,'id_measurement'].tolist()
            label_valid =  list(np.random.choice(label, int(len(label)*0.2)))
            
            label_train = [i for i in label if i not in label_valid]
            
            select[str(i)] = [label]
            
            
    return select

select= select_dict()

with open (src+'select.pkl','wb') as handle:
    pcikle.dump(select,handle)

In [0]:
id_list = np.unique(train_df['id_measurement'].tolist())
id_list

array([   0,    1,    2, ..., 2901, 2902, 2903])

In [0]:
idx_1 = train_df.loc[train_df['target']==1,'signal_id'].tolist()
idx_0 = train_df.loc[train_df['target']==0,'signal_id'].tolist()

print (len(idx_1)+len(idx_0))

8712


In [0]:
test_ratio = 0.2
valid_size_1 = int(525*test_ratio)
valid_size_0 = int(8187 * test_ratio)

print (valid_size_1, valid_size_0)

105 1637


In [0]:
valid_idx_1 = np.random.choice(idx_1, size= valid_size_1,replace=False)
train_idx_1 = [i for i in idx_1 if i not in valid_idx_1]

valid_idx_0 = np.random.choice(idx_0, size= valid_size_0,replace=False)
train_idx_0 = [i for i in idx_0 if i not in valid_idx_0]

print (len(valid_idx_1), len(train_idx_1) , len(valid_idx_0), len(train_idx_0))

105 420 1637 6550


In [0]:
def save_file_to(i):
  if i in train_idx_1:
    save_to = src_train_idx_1
  elif i in train_idx_0:
    save_to = src_train_idx_0
  
  elif i in valid_idx_1:
    save_to = src_valid_idx_1

  elif i in valid_idx_0:
    save_to = src_valid_idx_0
    
  else:
    print (f'i = {i},   something wrong')
    
  return save_to 

In [0]:



def sample(signal, kernel_size):
  sampled = np.zeros((signal.shape[0],  signal.shape[1]//kernel_size))
  print ('sampledshape',sampled.shape)
  for i in range(signal.shape[1]//kernel_size):
    begin = kernel_size * i
    #print (f'begin = {begin}')
    end = min(kernel_size * (i + 1), signal.shape[1])
    #print (f'end = {end}')
    sampled[:,  i] = np.mean(signal[:,  begin:end], axis=1)
    #print (f'from begin {begin } to end {end} mean value {sampled[:, :, i] } selected ')
  return sampled


In [0]:

def signals_to_img(fname,window=100):

  #plt.imshow(FNAME)
  #plt.show()
  img = Image.open(fname).convert('RGB')
  #print (img.size)
  #img = img.resize((750,500))
  x = np.array(img)
  #print (x.shape)

  img.save(fname)


In [0]:

def make_fig(j,sampled):

  length = sampled.shape[0]


  for i in range(length):
    
    idx = i+j*72
    save_to = save_file_to(idx)


    fig = plt.figure(figsize=(8, 4))
    


    plt.xticks([])
    plt.yticks([])
    
    sns.despine(bottom=True, left=True)
    plt.plot(sampled[i, :],color='r')
    #plt.plot(sampled[i*11, :], label='Phase 1')
    #plt.plot(sampled[i*11, :], label='Phase 2')
    #plt.legend()
    #print ('j=', j, 'i=',i, 'save_to =', save_to)
    fname = save_to+'signal_img_'+str(i+j*72)+'.png'
    plt.savefig(fname, format='png')
    plt.close(fig)
    signals_to_img(fname)
    fig.clf()

    #plt.despine(left=True,bottom=True)
    #plt.show()

In [0]:

#train_df = pq.read_pandas('train.parquet').to_pandas()

for j in range(121):
  
  subset_train = pq.read_pandas('train.parquet', columns=[str(i+j*72) for i in range(72)]).to_pandas()
  
  subset_value = np.array(signals).T.reshape((72//3, 3, 800000))
  

  print (subset_value.shape)
  #sampled = sample(subset_value,100)
 
  make_fig(j,subset_value)
  if j % 3 == 0:
    
    print (f'batch {j} /18 ok ')
  
print ('total complted')

(484, 800000)
sampledshape (484, 8000)
(484, 800000)
sampledshape (484, 8000)
(484, 800000)
sampledshape (484, 8000)
batch 12 /18 ok 
(484, 800000)
sampledshape (484, 8000)
(484, 800000)
sampledshape (484, 8000)
(484, 800000)
sampledshape (484, 8000)
batch 15 /18 ok 
(484, 800000)
sampledshape (484, 8000)
(484, 800000)
sampledshape (484, 8000)
total complted


In [0]:
import os
len( os.listdir(src_train_idx_1)) + len( os.listdir(src_valid_idx_1))


525

In [0]:
len( os.listdir(src_train_idx_0)) + len( os.listdir(src_valid_idx_0))


8187

In [0]:
os.listdir(src)

['test.parquet',
 'sample_submission.csv',
 'metadata_test.csv',
 'metadata_train.csv',
 'train.parquet',
 'all.zip',
 'vsb_power_line_eda.ipynb',
 'train_test_data']

# Start preparing test data

In [0]:
!cp 'drive/My Drive/kaggle/vsb_power_line/data/test.parquet' .

In [0]:
subset_train = pq.read_pandas('test.parquet', columns=[str(i) for i in range(99)]).to_pandas()
test_df.shape

In [0]:
subset_test = pq.read_pandas('test.parquet', columns=[str(i*2000 + j + 8712) for j in range(2000)]).to_pandas()

In [0]:
def make_fig_test(j,sampled):

  length = sampled.shape[0]


  for i in range(length):
    
    idx = i+j*500+8712
    save_to = src_test


    fig = plt.figure(figsize=(8, 4))
    


    plt.xticks([])
    plt.yticks([])
    
    sns.despine(bottom=True, left=True)
    plt.plot(sampled[i, :],color='r')
    #plt.plot(sampled[i*11, :], label='Phase 1')
    #plt.plot(sampled[i*11, :], label='Phase 2')
    #plt.legend()
    #print ('j=', j, 'i=',i, 'save_to =', save_to)
    fname = save_to+'signal_img_'+str(i+j*500 + 8712)+'.png'
    plt.savefig(fname, format='png')
    plt.close(fig)
    signals_to_img(fname)
    fig.clf()

    #plt.despine(left=True,bottom=True)
    #plt.show()

In [0]:
#train_df = pq.read_pandas('train.parquet').to_pandas()

import os

for j in range(30,40,1):
  
  subset_test = pq.read_pandas('test.parquet', columns=[str(i+j*500 + 8712) for i in range(500)]).to_pandas()
  subset_value = np.array(subset_test.values).T
  print (subset_value.shape)
  sampled = sample(subset_value,100)
 
  make_fig_test(j,sampled)
  if j % 3 == 0:
    
    print (f'batch {j} /18 ok ')
    
    print ('j=',j, len(os.listdir(src_test)))
    print (os.listdir(src_test)[-2:])
  
print ('total complted')

(500, 800000)
sampledshape (500, 8000)
batch 30 /18 ok 
j= 30 15500
['signal_img_24210.png', 'signal_img_24211.png']
(500, 800000)
sampledshape (500, 8000)
(500, 800000)
sampledshape (500, 8000)
(500, 800000)
sampledshape (500, 8000)
batch 33 /18 ok 
j= 33 17000
['signal_img_25710.png', 'signal_img_25711.png']
(500, 800000)
sampledshape (500, 8000)
(500, 800000)
sampledshape (500, 8000)
(500, 800000)
sampledshape (500, 8000)
batch 36 /18 ok 
j= 36 18500
['signal_img_27210.png', 'signal_img_27211.png']
(500, 800000)
sampledshape (500, 8000)
(500, 800000)
sampledshape (500, 8000)
(500, 800000)
sampledshape (500, 8000)
batch 39 /18 ok 
j= 39 20000
['signal_img_28710.png', 'signal_img_28711.png']
total complted


In [0]:
import os
len(os.listdir(src_test))

20000

In [0]:
def make_fig_test(j,sampled):

  length = sampled.shape[0]


  for i in range(length):
    
    idx = i+j*20000+8712
    save_to = src_test


    fig = plt.figure(figsize=(8, 4))
    


    plt.xticks([])
    plt.yticks([])
    
    sns.despine(bottom=True, left=True)
    plt.plot(sampled[i, :],color='r')
    #plt.plot(sampled[i*11, :], label='Phase 1')
    #plt.plot(sampled[i*11, :], label='Phase 2')
    #plt.legend()
    #print ('j=', j, 'i=',i, 'save_to =', save_to)
    fname = save_to+'signal_img_'+str(i+j*20000 + 8712)+'.png'
    plt.savefig(fname, format='png')
    plt.close(fig)
    signals_to_img(fname)
    fig.clf()

    #plt.despine(left=True,bottom=True)
    #plt.show()

In [0]:
#train_df = pq.read_pandas('train.parquet').to_pandas()

for j in range(1,2,1):
  
  subset_test = pq.read_pandas('test.parquet', columns=[str(i+j*20000 + 8712) for i in range(337)]).to_pandas()
  subset_value = np.array(subset_test.values).T
  print (subset_value.shape)
  sampled = sample(subset_value,100)
 
  make_fig_test(j,sampled)
  if j % 3 == 0:
    
    print (f'batch {j} /18 ok ')
  
print ('total complted')

(337, 800000)
sampledshape (337, 8000)
total complted


In [0]:
import os
len(os.listdir(src_test))

20337

# Test set completed

In [0]:
test_df = pd.read_csv(src+'metadata_test.csv')
idx_list = test_df['signal_id'].tolist()



In [0]:
test_list = os.listdir(src_test)
test_list[-5:]

['signal_img_29044.png',
 'signal_img_29045.png',
 'signal_img_29046.png',
 'signal_img_29047.png',
 'signal_img_29048.png']

In [0]:
idx_list[-5:]

[29044, 29045, 29046, 29047, 29048]